In [72]:
import os
import subprocess
import gzip
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import random

In [43]:
# data_dir = "data/pfam"
# os.makedirs(data_dir, exist_ok=True)

# # download regions.tsv.gz files
# url = "ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/"
# wget_cmd = f"wget -nd -r -A '*.fasta.gz' -P {data_dir} {url}"
# subprocess.run(wget_cmd, shell=True)

In [68]:
f = gzip.open('data/pfam/Pfam-A.fasta.gz', 'rt')

d = {}

lines_to_read = 10**7

i = 0
for line in tqdm(f):
    if line.startswith('>'):
        fam = line.split()[-1].split(';')[0]
        if fam not in d.keys():
            d[fam] = []
    else:
        d[fam].append(line.strip())

    i+= 1
    if i > lines_to_read:
        break


0it [00:00, ?it/s]

In [2]:
from datasets.protein import PfamDataset
from encoder.protein_encoders import ProteinSetEncoder
from generator.protein_generator import Progen2Generator

In [3]:
pf = PfamDataset(tokenize=False, lines_to_read=10**4)

In [9]:
from torch.utils.data import DataLoader
import torch

In [16]:
dataloader = DataLoader(pf, batch_size=1)
encoder = ProteinSetEncoder(latent_dim=16).to(device='cuda')
generator = Progen2Generator(latent_dim=16, max_length=128).to(device='cuda')

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
for batch in dataloader:
    samples = {}
    for key, value in batch['samples'].items():
        if isinstance(value, torch.Tensor):
            samples[key] = value.to('cuda')
        else:
            samples[key] = value
    print(generator.sample(encoder(samples), return_seq=True))
    break

(tensor([[[ 3,  2,  4, 14, 21, 11,  7, 15,  8,  7, 17, 17, 20, 13, 13,  9, 26,
          13, 21, 14, 15, 14, 15,  5,  9, 14, 22, 14,  8,  9,  9, 19,  9,  8,
           9, 12, 17,  9,  9, 11, 11, 13,  8, 22, 22, 13, 10, 19, 20, 22,  9,
          19, 15, 11, 10, 21, 12, 19, 12, 28,  8,  8, 10, 14, 12, 21, 25,  5,
          14, 10, 21,  8,  7, 15, 11,  7,  5, 10, 21, 26,  8,  9,  5, 14, 12,
          14, 23, 10,  9, 20, 20, 17, 13, 13, 17,  7, 12, 10, 17, 17, 14,  9,
          15, 14, 14, 21, 23, 13, 11,  7,  8, 15, 10, 13,  9, 21, 10, 14,  5,
           8,  9, 11,  4,  2,  4, 12, 26]]], device='cuda:0'), [['12KRGCLDCNNQIIEWIRKLKLAEKSKDEEPEDEHNEEGGIDSSIFPQSEPLGFRHPHYDDFKHRVAKFRDCLGCAFRWDEAKHKTFEQQNIINCHFNNKELKKRTIGCDLFIERFKADEG22HW']])
